In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [129]:
from __future__ import print_function, division

from sympy import S, Expr, Mul,Add, Pow, symbols, Function, Derivative, simplify
from sympy.physics.quantum import Operator
from sympy.printing.latex import print_latex
from sympy.printing.pretty.stringpict import prettyForm


from sympy import S, Expr, Mul, Add, Pow, Rational
from sympy.printing.pretty.stringpict import prettyForm

from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum.operator import Operator

from sympy.core.decorators import _sympifyit, call_highest_priority
from sympy.core.function import UndefinedFunction

from termcolor import colored

In [3]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

x, px, y, py = symbols("x px y py")

half = Rational(1, 2)

# Reduced class for debugging

In [144]:
class pb(Expr):
    _is_commutative = False
#     def __init__(self, A,B,debug=True, **kwargs):
#         self.A =A
#         self.B =B
#         self.order = None
#         self.indep_coords = kwargs.get("coords",None)
#         self.indep_mom = kwargs.get("mom",None)
#         print(self.indep_coords, self.indep_mom)
#         self.debug = debug
    
    def __new__(cls,A,B,debug=True,**kwargs):
        r = cls.eval(A,B,debug=debug)#,**kwargs)
        print("r is ",r)
        try:
            r.args[1].indep_coords = kwargs.get("coords",None)
            r.args[1].indep_mom    = kwargs.get("mom",None)
            print("var r is ",vars(r.args[1]))
            
        except:
            pass
        if r is not None:
            if debug:
                print("Eval returned a value")
            return r
        obj = Expr.__new__(cls,A,B)
        obj.A = A
        obj.B = B
        obj.indep_coords = kwargs.get("coords",None)
        obj.indep_mom    = kwargs.get("mom",None)
        obj.order = 1
        obj.debug = debug
        print("in obj")
        print(obj)
        print(vars(obj))
        print("obj kwargs : ",kwargs)
#         print(obj.doit())
        return obj
    
    @classmethod
    def eval(cls,a,b,debug):#, **kwargs):
        print("doing eval")
        if isinstance(a,UndefinedFunction):
            tmp = symbols("tmp")
            if isinstance(b, UndefinedFunction):
                if a(tmp).compare(b(tmp)) ==1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
#                     nob = cls(b,a,**kwargs)
                    nob = cls(b,a)
#                     nob=pb(b,a,**kwargs)
                    nob.order = -1
                    print(nob.indep_coords)
                    return S.NegativeOne*nob
            else:
                if a(tmp).compare(b) == 1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
#                     nob = cls(b,a, **kwargs)
                    nob = cls(b,a)
#                     nob=pb(b,a,**kwargs)
                    nob.order = -1
                    print(nob.indep_coords)
                    return S.NegativeOne*nob
        elif isinstance(b, UndefinedFunction):
            tmp = symbols("tmp")
            if a.compare(b(tmp)) ==1:
                if debug:
                    print("Interchanging order of arguments and put minus sign in front of bracket.")
#                 nob = cls(b,a,**kwargs)
                nob = cls(b,a)
#                 nob=pb(b,a,**kwargs)
                nob.order = -1
                print(nob.indep_coords)
                return S.NegativeOne*nob
        else:
            print("all are defined")
            if a.compare(b) ==1:
                print(vars(a))
                print(vars(b))
                if debug:
                    print("Interchanging order of arguments and put minus sign in front of bracket.")
#                 print("nob creation kwargs :",kwargs)
                nob = cls(b,a)
#                 nob = cls(b,a,**kwargs) #Expr.__new__(cls,B,A)#
#                 nob.A = b
#                 nob.B = a
#                 nob.indep_coords = kwargs.get("coords",None)
#                 nob.indep_mom    = kwargs.get("mom",None)
#                         nob=pb(b,a,**kwargs)
                nob.order = -1
#                 print(nob.indep_coords, nob.indep_mom)
#                 print(vars(nob))
#                 print(vars((S.NegativeOne*nob).args[1]))
#                 h =nob
                print()
                print(colored(vars(nob),"red"))
#                 print(vars(h.args[1]))
#                 print(vars((-h).args[1]))
#                 print(vars((S.NegativeOne * h).args[1]))
                hh = S.NegativeOne * nob
#                 print(hh.doit())
#                 hh.args[1].indep_coords= nob.indep_coords
#                 hh.args[1].indep_mom = nob.indep_mom
#                 print(vars(hh.args[1]))
#                 print(hh.doit())
                print("returning", hh)
                print("returning vars ", )
                return hh
#             else:
#                 nob = cls(b,a,**kwargs)
#                 nob.order = 1
#                 return nob

                
    @property
    def expr(self):
        if (self.indep_coords and self.indep_mom):
            hp = S(0)
            for qi, pi in zip(self.indep_coords, self.indep_mom):
                hp += self.A.diff(qi) * self.B.diff(pi)
                hp -= self.A.diff(pi) * self.B.diff(qi)
            return hp
        else:
            return self
        
    def _eval_derivative(self, symbol):
        new_expr = self.expr.diff(symbol)
        return new_expr
    
    def doit(self, debug=True, **hints):
        """ Evaluate commutator """
        A = self.args[0]
        B = self.args[1]
        order = self.order
        
        if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
            print("One is undefined function")
                    
            h = self
            return h
          
        elif isinstance(A,Function) or isinstance(B,Function):
            print("in Function")
            try:
                print(vars(self))
                print("expr:",self.expr)
                return self.expr

            except RecursionError:
                print("Check coords and mom if you expected this to evaluate.")
                return self
            
            return S.NegativeOne * PoissonBracket(A,B,coords=self.indep_coords,mom=self.indep_mom).expr
        
    def print_instance_attributes(self):
        for attribute, value in self.__dict__.items():
            print(attribute, '=', value)
        print(dir(self))

In [145]:
tpb1 = pb(B(x,px),A(x,px),coords=[x],mom=[px])

doing eval
all are defined
{'nargs': Naturals0}
{'nargs': Naturals0}
Interchanging order of arguments and put minus sign in front of bracket.
doing eval
all are defined
r is  None
in obj
pb(A(x, px), B(x, px))
{'A': A(x, px), 'B': B(x, px), 'indep_coords': None, 'indep_mom': None, 'order': 1, 'debug': True}
obj kwargs :  {}

{'A': A(x, px), 'B': B(x, px), 'indep_coords': None, 'indep_mom': None, 'order': -1, 'debug': True}
returning -pb(A(x, px), B(x, px))
returning vars 
r is  -pb(A(x, px), B(x, px))
var r is  {'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': -1, 'debug': True}
Eval returned a value


In [146]:
tpb1.doit()

in Function
{'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': -1, 'debug': True}
expr: -Derivative(A(x, px), px)*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), px)


-(-Derivative(A(x, px), px)*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), px))

In [147]:
tpb1 = pb(A(x,px),B(x,px),coords=[x],mom=[px])

doing eval
all are defined
r is  None
in obj
pb(A(x, px), B(x, px))
{'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': 1, 'debug': True}
obj kwargs :  {'coords': [x], 'mom': [px]}


In [148]:
tpb2 = pb(tpb1,C(x,px),coords=[x],mom=[px])

doing eval
all are defined
{'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': 1, 'debug': True}
{'nargs': Naturals0}
Interchanging order of arguments and put minus sign in front of bracket.
doing eval
all are defined
r is  None
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))
{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': 1, 'debug': True}
obj kwargs :  {}

{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': -1, 'debug': True}
returning -pb(C(x, px), pb(A(x, px), B(x, px)))
returning vars 
r is  -pb(C(x, px), pb(A(x, px), B(x, px)))
var r is  {'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': [x], 'indep_mom': [px], 'order': -1, 'debug': True}
Eval returned a value


In [149]:
tpb2.doit()

in Function
{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': [x], 'indep_mom': [px], 'order': -1, 'debug': True}
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))


-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))

In [150]:
tpb1 = pb(A(x,px),B(x,px))

doing eval
all are defined
r is  None
in obj
pb(A(x, px), B(x, px))
{'A': A(x, px), 'B': B(x, px), 'indep_coords': None, 'indep_mom': None, 'order': 1, 'debug': True}
obj kwargs :  {}


In [151]:
tpb2 = pb(tpb1,C(x,px))

doing eval
all are defined
{'A': A(x, px), 'B': B(x, px), 'indep_coords': None, 'indep_mom': None, 'order': 1, 'debug': True}
{'nargs': Naturals0}
Interchanging order of arguments and put minus sign in front of bracket.
doing eval
all are defined
r is  None
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))
{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': 1, 'debug': True}
obj kwargs :  {}

{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': -1, 'debug': True}
returning -pb(C(x, px), pb(A(x, px), B(x, px)))
returning vars 
r is  -pb(C(x, px), pb(A(x, px), B(x, px)))
var r is  {'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': -1, 'debug': True}
Eval returned a value


In [152]:


tpb2.doit() 

in Function
{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': None, 'indep_mom': None, 'order': -1, 'debug': True}
expr: pb(C(x, px), pb(A(x, px), B(x, px)))


-pb(C(x, px), pb(A(x, px), B(x, px)))

In [38]:
tpb1 = pb(A(x,px),B(x,px),coords=[x],mom=[px])

doing eval
all are defined
r is  None
in obj
pb(A(x, px), B(x, px))


In [34]:
tpb2 = pb(tpb1,C(x,px),coords=[x],mom=[px])

doing eval
all are defined
{'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': 1, 'debug': True}
{'nargs': Naturals0}
Interchanging order of arguments and put minus sign in front of bracket.
doing eval
all are defined
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))

doing eval
all are defined
in obj
pb(A(x, px), B(x, px))
doing eval
all are defined
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))
-pb(C(x, px), pb(A(x, px), B(x, px)))
{'A': C(x, px), 'B': pb(A(x, px), B(x, px)), 'indep_coords': [x], 'indep_mom': [px], 'order': -1, 'debug': True}
doing eval
all are defined
in obj
pb(A(x, px), B(x, px))
doing eval
all are defined
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))
-pb(C(x, px), pb(A(x, px), B(x, px)))
Eval returned a value


In [19]:
tpb1 = pb(A(x,px),B(x,px),coords=[x],mom=[px])
tpb2 = pb(tpb1,C(x,px),coords=[x],mom=[px])
print(tpb2.doit())

doing eval
all are defined
in obj
pb(A(x, px), B(x, px))
in Function
expr: -Derivative(A(x, px), px)*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), px)
-Derivative(A(x, px), px)*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), px)
doing eval
all are defined
{'A': A(x, px), 'B': B(x, px), 'indep_coords': [x], 'indep_mom': [px], 'order': 1, 'debug': True}
{'nargs': Naturals0}
Interchanging order of arguments and put minus sign in front of bracket.
doing eval
all are defined
in obj
pb(C(x, px), pb(A(x, px), B(x, px)))
in Function
expr: -Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Deriv

In [7]:
vars(tpb2.args[1])

{'A': C(x, px),
 'B': pb(A(x, px), B(x, px)),
 'indep_coords': [x],
 'indep_mom': [px],
 'order': -1,
 'debug': True}

In [113]:
tpb1 = pb(A,B)
tpb2 = pb(tpb1,C)


doing eval
in obj
{'is_commutative': False, 'is_complex': False, 'is_odd': False, 'is_extended_positive': False, 'is_extended_nonnegative': False, 'is_even': False, 'is_extended_nonpositive': False, 'is_algebraic': False, 'is_nonpositive': False, 'is_nonnegative': False, 'is_real': False, 'is_imaginary': False, 'is_transcendental': False, 'is_negative': False, 'is_integer': False, 'is_extended_nonzero': False, 'is_zero': False, 'is_composite': False, 'is_nonzero': False, 'is_noninteger': False, 'is_rational': False, 'is_positive': False, 'is_irrational': False, 'is_prime': False, 'is_extended_negative': False, 'is_extended_real': False, '_kwargs': {'commutative': False, 'complex': False, 'odd': False, 'extended_positive': False, 'extended_nonnegative': False, 'even': False, 'extended_nonpositive': False, 'algebraic': False, 'nonpositive': False, 'nonnegative': False, 'real': False, 'imaginary': False, 'transcendental': False, 'negative': False, 'integer': False, 'extended_nonzero': Fal

In [118]:
pb1= pb(A,B, coords=[x], mom=[px]);

doing eval
in obj


In [120]:
vars(S.NegativeOne*pb1)

TypeError: vars() argument must have __dict__ attribute

In [38]:
tpb2

-pb(C, pb(A, B))

In [39]:
tpb2.doit()

One is undefined function


-pb(C, pb(A, B))

# Full class

In [62]:
class PoissonBracket(Expr):
    """
    Operator class for the Poisson Bracket - allows to have it as a symbolic operator
    """
    
#     _op_priority = 11.0
    _is_commutative = False
    
    def __new__(cls,A,B,debug=True,**kwargs):
        # before return the object, do basic
        # testing and formatting of the bracket.
        # - orders arguments and updated sign
        # - returns S.Zero if:
        #    * one of the argumnents is missing
        #    * the arguments are equal
        #    * one of the arguments is a sympy number
        r = cls.eval(A,B,debug=debug,**kwargs)
        
        # if a valid object is returned from eval
        # than return this object
        if r is not None:
            if debug:
                print("Eval returned a value")
            return r
        
        # if eval has not returned anything
        # create a new object and return that one.
        obj = Expr.__new__(cls,A,B)
        obj.A = A
        obj.B = B
        
        # if independent coordinates and momenta are
        # provided store them in the object
        obj.indep_coords = kwargs.get("coords",None)
        obj.indep_mom    = kwargs.get("mom",None)
        
        # store if the ordering has been reversed
        # this is used in the doit method, otherwise
        # the sign is wrong.
        obj.order = 1
        
        # set debug to true to get intermediate outputs
        # for code debugging
        obj.debug = debug
        return obj
    
    @classmethod
    def eval(cls,a,b,debug, **kwargs):
        # check if both arguments of the
        # bracket are given otherwise return Zero
        if not (a and b):
            if debug:
                print("Missing term")
            return S.Zero
        
        # check if both arguments of the bracket
        # are equal, if yes return Zero
        if a==b:
            if debug:
                print("Bracket of equal aruments is Zero.")
            return S.Zero
        
        # check if one of the bracket arguments is
        # a sympy number, if yes return Zero.
        if a.is_number or b.is_number:
            if debug:
                print("At least one argument is a number, bracket is Zero")
            return S.Zero
        
        # order the elements
        if isinstance(a,UndefinedFunction):
            tmp = symbols("tmp")
            if isinstance(b, UndefinedFunction):
                if a(tmp).compare(b(tmp)) ==1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
                    nob = cls(b,a,**kwargs)
                    nob.order = -1
                    return -nob
            else:
                if a(tmp).compare(b) == 1:
                    if debug:
                        print("Interchanging order of arguments and put minus sign in front of bracket.")
                    nob = cls(b,a, **kwargs)
                    nob.order = -1
                    return -nob
        elif isinstance(b, UndefinedFunction):
            tmp = symbols("tmp")
            if a.compare(b(tmp)) ==1:
                if debug:
                    print("Interchanging order of arguments and put minus sign in front of bracket.")
                nob = cls(b,a,**kwargs)
                nob.order = -1
                return -nob
        else:
                if a.compare(b) ==1:
                        if debug:
                            print("Interchanging order of arguments and put minus sign in front of bracket.")
                        nob = cls(b,a,**kwargs)
                        nob.order = -1
                        return -nob
            
                
    @property
    def free_symbols(self):
        return self.A.free_symbols.union(self.B.free_symbols)
    
    @property
    def expr(self):
#         print(self.indep_coords)
        if (self.indep_coords and self.indep_mom):
            hp = S(0)
            for qi, pi in zip(self.indep_coords, self.indep_mom):
                hp += self.A.diff(qi) * self.B.diff(pi)
                hp -= self.A.diff(pi) * self.B.diff(qi)
            return hp
        else:
            return self
        
    def _eval_derivative(self, symbol):
        """
        Necessary to get the full expanded expression
        when one of the arguments is itself a PoissonBracket.
        In other words to allow evalutation of nested Poisson Brackets.
        """
        new_expr = self.expr.diff(symbol)
        return new_expr
    
    def doit(self, debug=True, **hints):
        """ Evaluate commutator """
        A = self.args[0]
        B = self.args[1]
        order = self.order
        if debug:
                print(self)
                print(self.__class__)
                print(self.args)
                try:
                    print(self.expr)
                except:
                    print("expr failed")
                    pass
                
        if self.order == 1:
            if debug:
                print("order 1")
            if isinstance(A, Operator) and isinstance(B, Operator):
                try:
                    comm = A._eval_commutator(B, **hints)
                except NotImplementedError:
                    try:
                        comm = -1*B._eval_commutator(A, **hints)
                    except NotImplementedError:
                        comm = None
                if comm is not None:
                    return comm.doit(**hints)
                
            if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
                if debug:
                    print("One is undefined function")
                return self
            
            elif isinstance(A,Function) or isinstance(B,Function):
                if debug:
                    print("in Function")
                try:
                    print("expr:",self.expr)
                    return self.expr
                except RecursionError:
                    print("Check coords and mom if you expected this to evaluate.")
                    return self
#                 h = self.expr 
#                 print(self.expr)
#                 return h
            
            return self.expr
        
        else:
            if debug:
                print("order -1")
                
            if isinstance(A, UndefinedFunction) or isinstance(B, UndefinedFunction):
                if debug:
                    print("One is undefined function")
                    print(self)
                h = self
                return h
          
            elif isinstance(A,Function) or isinstance(B,Function):
                if debug:
                    print("in Function")
                    
                try:
                    print("expr:",self.args)
#                     print(self.indep_coords)
                    h = self.expr
                    return h
                except RecursionError:
                    print("Check coords and mom if you expected this to evaluate.")
                    return self
                    
#                 h = self.expr 
#                 print("expr:",self.expr)
#                 if isinstance(A, PoissonBracket) : print(A.indep_coords);
#                 if isinstance(B, PoissonBracket) : print(B.indep_coords);
               
#                 return h
            
            return S.NegativeOne * PoissonBracket(A,B,coords=self.indep_coords,mom=self.indep_mom).expr
    
    def _eval_expand_commutator(self, **hints):
        A = self.A #self.args[0]
        B = self.B #self.args[1]

        if isinstance(A, Add):
            # [A + B, C]  ->  [A, C] + [B, C]
            sargs = []
            for term in A.args:
                comm = PoissonBracket(term, B)
                print(comm.__class__)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
#             print(sargs)
            return Add(*sargs)
        elif isinstance(B, Add):
            # [A, B + C]  ->  [A, B] + [A, C]
            sargs = []
            for term in B.args:
                comm = PoissonBracket(A, term)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
            return Add(*sargs)
        elif isinstance(A, Mul):
            # [A*B, C] -> A*[B, C] + [A, C]*B
            a = A.args[0]
            b = Mul(*A.args[1:])
            c = B
            comm1 = PoissonBracket(b, c)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(a, comm1)
            second = Mul(comm2, b)
#             print(first,second)
            return Add(first, second)
        elif isinstance(B, Mul):
            # [A, B*C] -> [A, B]*C + B*[A, C]
            a = A
            b = B.args[0]
            c = Mul(*B.args[1:])
            comm1 = PoissonBracket(a, b)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(comm1, c)
            second = Mul(b, comm2)
            return Add(first, second)
        elif isinstance(A, Pow):
            # [A**n, C] -> A**(n - 1)*[A, C] + A**(n - 2)*[A, C]*A + ... + [A, C]*A**(n-1)
            return self._expand_pow(A, B, 1)
        elif isinstance(B, Pow):
            # [A, C**n] -> C**(n - 1)*[C, A] + C**(n - 2)*[C, A]*C + ... + [C, A]*C**(n-1)
            return self._expand_pow(B, A, -1)
        return self
    
    def _expand_pow(self, A, B, sign):
        exp = A.exp
        if not exp.is_integer or not exp.is_constant() or abs(exp) <= 1:
            # nothing to do
            return self
        base = A.base
        if exp.is_negative:
            base = A.base**-1
            exp = -exp
        comm = PoissonBracket(base, B).expand(commutator=True)

        result = base**(exp - 1) * comm
        for i in range(1, exp):
            result += base**(exp - 1 - i) * comm * base**i
        return sign*result.expand()
    
    
    
    def _latex(self, printer, *args):
#         print(tuple([
#             printer._print(arg.func, *args) if arg.__class__.__name__ != "PoissonBracket" else  printer._print(arg, *args) for arg in self.args[:2] ]))
        news = []
        for arg in self.args[:2]:
#             print(type(arg))
            if isinstance(arg,Add):
                news.append(" + ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
            
            elif isinstance(arg,PoissonBracket):
                news.append(printer.doprint(arg,*args))
#                 news.append("\\lbrace %s,%s\\rbrace" %tuple([
#             printer.doprint(a.func, *args) if a.is_Function and not isinstance(a,UndefinedFunction)  else  printer.doprint(a, *args) for a in arg.args[:2] ]))
            
            elif isinstance(arg,Mul):
                 news.append(" * ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
#                 print([a for a in arg.args])
#                 news.append(" * ".join([ 
#                     printer._print(a.func, *args) 
#                     if a.is_Function  and not (isinstance(a,UndefinedFunction))
#                     else  printer._print(a, *args) for a in args
#                 ]))
            elif isinstance(arg,Pow):
                if isinstance(arg.args[0], UndefinedFunction):
                    news.append("{}^{}".format([printer.doprint(a) for a in arg.args]))
                elif isinstance(arg.args[0], Function):
                    news.append("{}^{}".format(printer.doprint(arg.args[0].func,*args), printer.doprint(arg.args[1],*args)))
                else:
                    news.append(printer.doprint(arg, *args))
                
            elif isinstance(arg, Function) and not (isinstance(arg,UndefinedFunction)):
                news.append(printer.doprint(arg.func, *args))
                                                    
            else: # isinstance(arg, Function) and isinstance(arg,UndefinedFunction):
                news.append(printer.doprint(arg, *args))
    
        print(news)
        return "\\lbrace %s,%s\\rbrace" %tuple(news)
        
        return "\\lbrace %s,%s\\rbrace" %tuple([
            printer._print(arg.func, *args) if arg.is_Function  and not (isinstance(arg,UndefinedFunction))
            else  printer._print(arg, *args) 
            for arg in self.args[:2] ])
#         return "\\lbrace %s,%s\\rbrace" % tuple([
#             printer._print(arg, *args) for arg in self.args[:2]])

# Tests Setup

In [63]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

x, px, y, py = symbols("x px y py")

half = Rational(1, 2)

# Tests

In [64]:
_file__ = "Untitled.ipynb"
import ipytest

import pytest
ipytest.autoconfig()

## Single Brackets

In [98]:
%%run_pytest[clean]

import pytest

single_pb_fail = [
    ((1,2)   , 0),
    ((1,half), 0),
]

single_pb_pass = [
    ((S.One   , half),{}, 0),
    ((A       , half),{}, 0),
    ((A(x)    , half),{}, 0),
    ((A(x,px) , half),{}, 0),
    ((S.One   , half),{"coords":[x]}, 0),
    ((A       , half),{"coords":[x]}, 0),
    ((A(x)    , half),{"coords":[x]}, 0),
    ((A(x,px) , half),{"coords":[x]}, 0),
    (reversed((S.One   , half)),{}, 0),
    (reversed((A       , half)),{}, 0),
    (reversed((A(x)    , half)),{}, 0),
    (reversed((A(x,px) , half)),{}, 0),
    (reversed((S.One   , half)),{"coords":[x]}, 0),
    (reversed((A       , half)),{"coords":[x]}, 0),
    (reversed((A(x)    , half)),{"coords":[x]}, 0),
    (reversed((A(x,px) , half)),{"coords":[x]}, 0),
]

single_pb_doit_fail = [
    
]

single_pb_doit_pass = [
    # Brackets doit returns self - no indep coords/mom to take der.
    ((A       , B       ),{}, PoissonBracket(A, B)),
    ((A(x,px) , B       ),{}, PoissonBracket(A(x,px), B)),
    ((A       , B(x,px) ),{}, PoissonBracket(A, B(x,px))),
    ((A(x,px) , B(x,px) ),{}, PoissonBracket(A(x,px), B(x,px))),
    ((x **2   , x + px  ),{}, PoissonBracket(x ** 2, x + px)),
    
    (reversed((A       , B       )),{}, PoissonBracket(B, A)),
    (reversed((A(x,px) , B       )),{}, PoissonBracket(B, A(x,px))),
    (reversed((A       , B(x,px) )),{}, PoissonBracket(B(x,px),A)),
    (reversed((A(x,px) , B(x,px) )),{}, PoissonBracket(B(x,px),A(x,px))),
    
    # Brackets actually performing the doit
    ((A       , B       ),{"coords":[x]}, PoissonBracket(A, B)),
    ((A(x,px) , B       ),{"coords":[x]}, PoissonBracket(A(x,px), B)),
    ((A       , B(x,px) ),{"coords":[x]}, PoissonBracket(A, B(x,px))),
    ((A(x,px) , B(x,px) ),{"coords":[x]}, PoissonBracket(A(x,px), B(x,px))),
    
    ((x **2     , x + px     ), {"coords":[x]  , "mom": [px]   }, 2*x),
    ((A(x, px)  , B(x, y, py)), {"coords":[x]  , "mom": [px]   }, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x,y, px), B(x, y, py)), {"coords":[x]  , "mom": [px]   }, -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x)),
    ((A(x, px)  , B(x, y, py)), {"coords":[x,y], "mom": [px]   }, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x, px)  , B(x, y, py)), {"coords":[x,y], "mom": [px,py]}, -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)),
    ((A(x,y, px), B(x, y, py)), {"coords":[x,y], "mom": [px,py]}, -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x) +
     Derivative(A(x, y, px), y)*Derivative(B(x, y, py), py)),
    
    ((A(x,px)+B(x,y,py), C(x,px)           ), {"coords":[x,y], "mom":[px,py]}, (Derivative(A(x, px), x) + Derivative(B(x, y, py), x))*Derivative(C(x, px), px) - Derivative(A(x, px), px)*Derivative(C(x, px), x) ),
    ((A(x,px)*B(x,y,py), C(x,px)           ), {"coords":[x,y], "mom":[px,py]}, (A(x, px)*Derivative(B(x, y, py), x) + Derivative(A(x, px), x)*B(x, y, py))*Derivative(C(x, px), px) - Derivative(A(x, px), px)*B(x, y, py)*Derivative(C(x, px), x)),
    ((A(x,px)           , B(x,y,py)*C(x,px)), {"coords":[x,y], "mom":[px,py]}, S.NegativeOne * ((B(x, y, py)*Derivative(C(x, px), x) + Derivative(B(x, y, py), x)*C(x, px))*Derivative(A(x, px), px) - B(x, y, py)*Derivative(C(x, px), px)*Derivative(A(x, px), x))),
    ((A(x,px)**3        , B(x,y,py)        ), {"coords":[x,y], "mom":[px,py]}, -3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)),
]
@pytest.mark.parametrize('input,expected',single_pb_fail)
def test_single_pb_parametrized_fail(input, expected):
    with pytest.raises(AttributeError):
        test = PoissonBracket(*input)
        

@pytest.mark.parametrize('input,kwargs,expected',single_pb_pass)
def test_single_pb_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test == expected
    
    
@pytest.mark.parametrize('input,kwargs,expected',single_pb_doit_pass)
def test_single_pb_doit_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test.doit() == expected

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
...................................... [ 95%]
..                                     [100%]
40 passed in 0.19s


## Double Brackets

In [66]:
%%run_pytest[clean]

import pytest

double_pb_fail = [
    
]

double_pb_pass = [
    ((S.One   , half), (S.One,) ,{}            , {}, 0),
    ((A       , half), (S.One,) ,{}            , {}, 0),
    ((A(x)    , half), (S.One,) ,{}            , {}, 0),
    ((A(x,px) , half), (S.One,) ,{}            , {}, 0),
    ((S.One   , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (S.One,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A,) ,{}            , {}, 0),
    ((A       , half), (A,) ,{}            , {}, 0),
    ((A(x)    , half), (A,) ,{}            , {}, 0),
    ((A(x,px) , half), (A,) ,{}            , {}, 0),
    ((S.One   , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x),) ,{}            , {}, 0),
    ((A       , half), (A(x),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x),) ,{}            , {}, 0),
    ((S.One   , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x),) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x,px),) ,{}            , {}, 0),
    ((A       , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{}            , {}, 0),
    ((S.One   , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A       , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A,) ,{}            , {}, 0),
    (reversed((A       , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    ((A     , B    ), (S.One,) ,{}            , {}, 0),
]

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_pass)
def test_double_pb_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    assert pb2 == expected

...................................... [ 58%]
...........................            [100%]
65 passed in 0.24s


In [67]:
%%run_pytest[clean]

import pytest

double_pb_doit_fail = [
   
]

double_pb_doit_pass = [
    ((A      , B      ), (S.One,), {}, {}, 0),
    ((A      , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A,B),C)),
    ((A(x,px), B(x,px)), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    ((x **2  , x + px ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(x**2,x+px),C)),
    
    ((A      , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A, B),C)),
    ((A(x)   , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A      , B(x)   ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A, B(x)), C)),
    ((A      , B      ), (C(x) ,), {}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A      , B      ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A, B),C)),
    ((A(x)   , B      ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A      , B(x)   ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A, B(x)), C)),
    ((A      , B      ), (C(x) ,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    
    ((A(x)  , B       ), (C    ,), {}, {"coords":[x]}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A     , B(x)    ), (C    ,), {}, {"coords":[x]}, PoissonBracket( PoissonBracket(A, B(x)),C )),
    ((A     , B       ), (C(x) ,), {}, {"coords":[x]}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    ((A     , B       ), (C    ,), {}, {"coords":[x]}, PoissonBracket(PoissonBracket(A, B), C)),
   
    
    ((A(x)   , B(x)   ), (C      ,), {"coords":[x]}, {}            , PoissonBracket(PoissonBracket(A(x), B(x)), C)),
    ((A(x)   , B(x)   ), (C(x)   ,), {"coords":[x]}, {}            , S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A(x), B(x)))),
    ((A(x)   , B(x)   ), (C(x)   ,), {}            , {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x]}, {}            , -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),

    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {"coords":[x], "mom": [px]},  -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, 0),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]},  S.NegativeOne * (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, S.NegativeOne *(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))),
    
    ((A(x,px),B),(C,), {}, {}, PoissonBracket(PoissonBracket(A(x, px), B), C)),
    ((x **2  , x + px ), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(x**2,x+px,coords=[x],mom=[px]),C)),
    ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C(x,px))),
    ((A(x,px), B(x,px)), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    
    ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]},S.NegativeOne *(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))),
    
]

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_doit_pass)
def test_double_pb_doit_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    print(pb2.doit()-expected)
    assert pb2.doit() == expected

# @pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_doit_fail)
# def test_double_pb_doit_parametrized_fail(input1, input2, kwargs1, kwargs2, expected):
#     with pytest.raises(AttributeError):
#         pb1 = PoissonBracket(*input1, **kwargs1)
#         pb2 = PoissonBracket(pb1, *input2, **kwargs2)

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value
Interchanging order of arguments and put minus sig

# Triple brackets

In [107]:
%%run_pytest[clean]

import pytest

triple_pb_doit_fail = [
   
]

triple_pb_doit_pass = [
    (
        (A(x,px), B(x,y,py)), 
        (C(x,px) ,),
        (D(x,y)  ,),
        {"coords":[x,y], "mom": [px,py]}, 
        {"coords":[x,y], "mom": [px,py]}, 
        {"coords":[x,y], "mom": [px,py]}, 
        (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), py, (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), py, x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), py, x))*Derivative(D(x, y), y) + (-Derivative(C(x, px), px)*(-Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), (px, 2), x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), (px, 2))*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 3))*Derivative(B(x, y, py), x) - Derivative(C(x, px), px, x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))*Derivative(D(x, y), x)
    ),
]

@pytest.mark.parametrize('input1,input2,input3,kwargs1,kwargs2,kwargs3,expected',triple_pb_doit_pass)
def test_double_pb_doit_parametrized_pass(input1, input2, input3, kwargs1, kwargs2, kwargs3, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    pb3 = PoissonBracket(pb2, *input3, **kwargs3)
    assert pb3.doit() == expected


.                                      [100%]
1 passed in 0.04s


# Individual test

In [103]:
input,kwargs, expected =  ((A(x,px)**3,B(x,y,py)), 
                             {"coords":[x,y], "mom":[px,py]}, 
                             -3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
                            )
test = PoissonBracket(*input,**kwargs)

print()
print("------------------------------")
print(test.doit())
print()
print(expected)
print("------------------------------")
print(simplify(test.doit()-expected))

assert test.doit() == expected

# pb2.free_symbols

del input,kwargs, expected,test


------------------------------
PoissonBracket(A(x, px)**3, B(x, y, py))
<class '__main__.PoissonBracket'>
(A(x, px)**3, B(x, y, py))
-3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
order 1
in Function
expr: -3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
-3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)

-3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
------------------------------
PoissonBracket(A(x, px)**3, B(x, y, py))
<class '__main__.PoissonBracket'>
(A(x, px)**3, B(x, y, py))
-3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
order 1
in Function
expr: -3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
0
PoissonBracket(A(x, px)**3, B(x, y, py))
<class '__main__.PoissonBracket'>
(A(x, px)**3, B(x, y, py))
-3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
or

In [99]:
input1,input2,kwargs1,kwargs2, expected =  ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, 
                                           S.NegativeOne *(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))
                                           )
pb1 = PoissonBracket(*input1, **kwargs1)
pb2 = PoissonBracket(pb1, *input2, **kwargs2)
print()
print("------------------------------")
print(pb2.doit())
print()
print(expected)
print("------------------------------")
print(simplify(pb2.doit()-expected))

assert pb2.doit() == expected

# pb2.free_symbols

del input1,input2,kwargs1,kwargs2, expected, pb1, pb2

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value

------------------------------
PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, px)))
<class '__main__.PoissonBracket'>
(C(x, px), PoissonBracket(A(x, px), B(x, px)))
-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))
order -1
in Function
expr: (C(x, px), PoissonBracket(A(x, px), B(x, px)))
-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), 

In [106]:
input1,input2,input3, kwargs1,kwargs2, kwargs3, expected =  (
    (A(x,px), B(x,y,py)), 
    (C(x,px) ,),
    (D(x,y)  ,),
    {"coords":[x,y], "mom": [px,py]}, 
    {"coords":[x,y], "mom": [px,py]}, 
    {"coords":[x,y], "mom": [px,py]}, 
    (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), py, (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), py, x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), py, x))*Derivative(D(x, y), y) + (-Derivative(C(x, px), px)*(-Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), (px, 2), x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), (px, 2))*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 3))*Derivative(B(x, y, py), x) - Derivative(C(x, px), px, x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x))*Derivative(D(x, y), x)
)
pb1 = PoissonBracket(*input1, **kwargs1)
pb2 = PoissonBracket(pb1, *input2, **kwargs2)
pb3 = PoissonBracket(pb2, *input3, **kwargs3)
print()
print("------------------------------")
print(pb3.doit())
print()
print(expected)
print("------------------------------")
print(simplify(pb2.doit()-expected))

assert pb3.doit() == expected

# pb2.free_symbols

del input1,input2, input3, kwargs1,kwargs2, kwargs3,  expected, pb1, pb2, pb3

Interchanging order of arguments and put minus sign in front of bracket.
Eval returned a value

------------------------------
PoissonBracket(-PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py))), D(x, y))
<class '__main__.PoissonBracket'>
(-PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, y, py))), D(x, y))
(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), py, (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), py, x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), py, x))*Derivative(D(x, y), y) + (-Derivative(C(x, px), px)*(-Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), (px, 2), x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), (px, 2))*(-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 3))*Derivative(B(x, y, py), x) - Derivative(C(x, px),